# G2


In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def objective_function_g2(x):
    x = np.array(x).reshape(-1, 7)
    x1, x2, x3, x4, x5, x6, x7 = x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4], x[:, 5], x[:, 6]
    
    f = (x1 - 10)**2 + 5*(x2 - 12)**2 + x3**4 + 3*(x4 - 11)**2 + 10*x5**6 + 7*x6**2 + x7**4 - 4*x6*x7 - 10*x6 - 8*x7
    return f

def constraint_function_g2(x):
    x = np.array(x).reshape(-1, 7)
    x1, x2, x3, x4, x5, x6, x7 = x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4], x[:, 5], x[:, 6]
    
    g1 = -127 + 2*x1**2 + 3*x2**4 + x3 + 4*x4**2 + 5*x5
    g2 = -282 + 7*x1 + 3*x2 + 10*x3**2 + x4 - x5
    g3 = -196 + 23*x1 + x2**2 + 6*x6**2 - 8*x7
    g4 = 4*x1**2 + x2**2 - 3*x1*x2 + 2*x3**2 + 5*x6 - 11*x7

    return np.vstack((g1, g2, g3, g4))

def constraints_g2(x):

    return np.all(constraint_function_g2(x) <= 0, axis=0)

def objective_function_with_constraints_g2(x):
    if not constraints_g2(x):
       
        return 0
    else:
        
        return objective_function_g2(x)




## PSO 

In [2]:
import numpy as np
import pandas as pd

# Número de partículas
n_particles = 20

# Número de dimensões
n_dimensions = 7

# Limites inferiores e superiores para cada dimensão
lb = [-10]*7 
ub = [10]*7 

# Parâmetros do PSO
c1 = 2
c2 = 2
w = 0.7
max_iter = 300
n_experiments = 30

# Armazenar resultados
results = []
pso_final =[]
 
for exp in range(n_experiments):
    #Inicializar enxame
    particles = np.empty((n_particles, n_dimensions))
    for i in range(n_particles):
        while True:
            particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
            if objective_function_with_constraints_g2(particle) != 0:
                particles[i] = particle
                break
    
    personal_best_scores = np.array([objective_function_with_constraints_g2(p) for p in particles])
    print(personal_best_scores)
    velocities = np.zeros((n_particles, n_dimensions))
    personal_best_positions = particles.copy()

    # Inicializar o melhor global
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = min(personal_best_scores)
    print('Global best score: ',global_best_score)
    # Loop principal
    for i in range(max_iter):
        print(f'------------------')
        print(f'experimento {exp+1} de {n_experiments}')
        print(f'iteração {i+1}')
        print(f'Global best score: {global_best_score}')
        for j in range(n_particles):
            # Atualizar velocidade
            velocities[j] = (w * velocities[j] +
                             c1 * np.random.rand() * (personal_best_positions[j] - particles[j]) +
                             c2 * np.random.rand() * (global_best_position - particles[j]))
            
            # Atualizar posição
            particles[j] += velocities[j]
            
            # Aplicar limites
            particles[j] = np.clip(particles[j], lb, ub)

            # Verificar restrições
            if not constraints_g2(particles[j]):
                # A nova posição não satisfaz todas as restrições.
                # Trate isso de maneira adequada. Por exemplo:
                particles[j] = personal_best_positions[j].copy()
            
            # Atualizar o melhor pessoal
            score = objective_function_with_constraints_g2(particles[j])
            if score < personal_best_scores[j]:
                personal_best_positions[j] = particles[j].copy()
                personal_best_scores[j] = score
                
                # Atualizar o melhor global
                if score < global_best_score:
                    global_best_position = particles[j].copy()
                    global_best_score = score

        # Armazenar resultados
        results.append({
            'Experiment': exp+1,
            'Iteration': i+1,
            'Best Fitness': global_best_score,
            'Best Particle': global_best_position,
            'Worst Fitness': max(personal_best_scores),
            'Average Fitness': np.mean(personal_best_scores),
            'Standard Deviation': np.std(personal_best_scores),
            'Median Fitness': np.median(personal_best_scores),
        })
    pso_final.append(np.min(global_best_score))        

# Criar dataframe com os resultados
df = pd.DataFrame(results)

df.head(5)

Streaming output truncated to the last 5000 lines.
------------------
experimento 11 de 30
iteração 272
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 273
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 274
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 275
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 276
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 277
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 278
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 279
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 280
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 281
Global best score: [1359.30621326]
------------------
experimento 11 de 30
iteração 282
Global best scor

KeyboardInterrupt: ignored

In [ ]:
df.head(1000)

In [ ]:
df_pso = df.sort_values('Best Fitness', ascending=True).reset_index()
best_PSO_individual = df_pso.loc[0, 'Best Particle']
fitness_best_PSO_individual = df_pso.loc[0, 'Best Fitness']
df_pso.head()

## GA

In [4]:
# Definindo os parâmetros do algoritmo genético
n_particles = 50
pop_size = 50
n_dimensions = 7
genes= 7
generations = 500
lb = [-10]*7
ub = [10]*7
crossover_prob = 0.6
mutation_prob = 0.2
runs = 30  # Número de execuções

results_ga = []
ga_final = []

alpha = 0.5  # Este é o parâmetro alfa, que determina o tamanho do intervalo de blend

for run in range(runs):

    # Inicialização da população
    particles = np.empty((n_particles, n_dimensions))
    for i in range(n_particles):
        while True:
            particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
            if objective_function_with_constraints_g2(particle) != 0:
                particles[i] = particle
                break
    population = particles

    # Loop das gerações
    for gen in range(generations):
        
        # Avaliação da população
        fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
        print(f'------------------')
        print(f'experimento {run+1} de {runs}')
        print(f'generation {gen+1}')
        print(f'Global best score: {min(fitness_values)}')
        print(f'Global worst score: {max(fitness_values)}')

        # Seleção por torneio
        parents = []
        for _ in range(pop_size):
            contenders_indices = np.random.choice(len(population), size=2)
            contenders = [population[i] for i in contenders_indices]
            fitness_contenders = [fitness_values[i] for i in contenders_indices]
            parents.append(contenders[np.argmin(fitness_contenders)])

        # Assegurando que o melhor indivíduo sempre estará na próxima geração
        best_individual = tuple(population[np.argmin(fitness_values)])
        if best_individual not in [tuple(parent) for parent in parents]:
            parents[0] = np.array(best_individual)

        # Crossover
        offspring = []
        for i in range(pop_size//2):
            parent1 = parents[i * 2]
            parent2 = parents[i * 2 + 1]
            if np.random.rand() <= crossover_prob:
                # Blend Crossover
                d = abs(parent1 - parent2)
                low = np.minimum(parent1, parent2) - alpha * d
                high = np.maximum(parent1, parent2) + alpha * d
                child1 = np.random.uniform(low, high)
                child2 = np.random.uniform(low, high)
                child1, child2 = np.clip(child1, lb, ub), np.clip(child2, lb, ub)

                # Uniform Crossover
                mask = np.random.randint(0, 2, size=genes).astype(np.bool)
                child1_u, child2_u = parent1.copy(), parent2.copy()
                child1_u[mask], child2_u[mask] = parent2[mask], parent1[mask]
                child1_u, child2_u = np.clip(child1_u, lb, ub), np.clip(child2_u, lb, ub)
                # Adicionando à lista de filhos apenas se satisfizerem as restrições
                children = [child1, child2, child1_u, child2_u]
                for child in children:
                    if objective_function_with_constraints_g2(child) != 0:  # Verificando as restrições
                        offspring.append(child)

        # Substituição dos piores indivíduos pelos filhos
        tamanho_substituicao = min(len(offspring), len(population))
        indices_substituicao = np.random.choice(len(population), size=tamanho_substituicao, replace=False)
        for i, index in enumerate(indices_substituicao):
            population[index] = offspring[i]

        # Mutação uniforme
        best_index = np.argmin(fitness_values)  # Encontre o índice do melhor indivíduo
        for i in range(len(population)):
            if i != best_index and np.random.rand() <= mutation_prob:  # Não mutar o melhor indivíduo
                mutation_point = np.random.randint(genes)
                new_value = np.random.uniform(lb[mutation_point], ub[mutation_point])
                temp_individual = population[i].copy()
                temp_individual[mutation_point] = new_value
                if objective_function_with_constraints_g2(temp_individual) != 0:  # Aceite a mutação apenas se o indivíduo mutado satisfizer as restrições
                    population[i] = temp_individual

        # Avaliação da população
        fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]

        # Estatísticas da geração
        fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
        best_fitness = min(fitness_values)
        worst_fitness = max(fitness_values)
        mean_fitness = np.mean(fitness_values)
        std_dev_fitness = np.std(fitness_values)

        results_ga.append({
            'run': run,
            'geracao': gen,
            'melhor_fitness': best_fitness,
            'melhor_particula': population[np.argmin(fitness_values)],
            'pior_fitness': worst_fitness,
            'media_dos_fitness': mean_fitness,
            'desvio_padrao': std_dev_fitness,
            'n_de_individuos': len(population),
        })
    ga_final.append(best_fitness)

df_ga = pd.DataFrame(results_ga)
df_ga.head(5)


               


------------------
experimento 1 de 30
generation 1
Global best score: [1219.01755385]
Global worst score: [9154730.55785742]
------------------
experimento 1 de 30
generation 2
Global best score: [1210.22287942]
Global worst score: [6254389.13872762]
------------------
experimento 1 de 30
generation 3
Global best score: [1286.11676372]
Global worst score: [1320981.17690098]
------------------
experimento 1 de 30
generation 4
Global best score: [782.27376712]
Global worst score: [6188714.05088963]
------------------
experimento 1 de 30
generation 5
Global best score: [812.87442621]
Global worst score: [6247534.30233024]
------------------
experimento 1 de 30
generation 6
Global best score: [751.38856054]
Global worst score: [229293.85093956]
------------------
experimento 1 de 30
generation 7
Global best score: [885.7660552]
Global worst score: [39973.18033187]


<ipython-input-4-897128653526>:69: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.random.randint(0, 2, size=genes).astype(np.bool)


Streaming output truncated to the last 5000 lines.
------------------
experimento 29 de 30
generation 1
Global best score: [910.12031719]
Global worst score: [9538955.32577587]
------------------
experimento 29 de 30
generation 2
Global best score: [1108.63912536]
Global worst score: [9538955.32577587]
------------------
experimento 29 de 30
generation 3
Global best score: [1828.25504495]
Global worst score: [9538955.32577587]
------------------
experimento 29 de 30
generation 4
Global best score: [1787.59445165]
Global worst score: [9538955.32577587]
------------------
experimento 29 de 30
generation 5
Global best score: [1576.05951466]
Global worst score: [7896071.23420708]
------------------
experimento 29 de 30
generation 6
Global best score: [1023.52461089]
Global worst score: [1655875.13173704]
------------------
experimento 29 de 30
generation 7
Global best score: [1194.40408124]
Global worst score: [830174.13294521]
------------------
experimento 29 de 30
generation 8
Global be

,run,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,0,0,[1210.2228794244668],"[1.5607499149436561, 1.584179208727668, -0.600...",[6254389.1387276165],460455.347825,1.052915e+06,50
1,0,1,[1286.1167637207518],"[1.560749914943653, 1.584179208727668, -0.6008...",[1320981.1769009787],57849.866084,2.308636e+05,50
2,0,2,[782.2737671242214],"[1.5607499149436552, 1.584179208727668, -0.600...",[6188714.050889629],174094.726846,8.992035e+05,50
3,0,3,[812.8744262060745],"[1.560749914943653, 1.584179208727668, -0.6008...",[6247534.302330235],126577.655084,8.744228e+05,50
4,0,4,[751.3885605377521],"[1.560749914943657, 1.584179208727668, -0.6008...",[229293.85093956068],6006.296631,3.191631e+04,50


In [5]:
df_ga = df_ga.sort_values('melhor_fitness', ascending=True).reset_index()
best_ga_individual = df_ga.loc[0, 'melhor_particula']
fitness_best_ga_individual = df_ga.loc[0, 'melhor_fitness']
df_ga.head()

,index,run,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,775,1,275,[683.3944377490479],"[2.343074192693262, 1.9856719274571704, -0.583...",[11850.418006945594],974.056255,1613.528941,50
1,773,1,273,[683.3947901731334],"[2.343074192693261, 1.9856719274571741, -0.583...",[1802.2587890186546],705.773721,156.640724,50
2,772,1,272,[683.3949746617823],"[2.34307419269326, 1.9856719274571741, -0.5831...",[2423.130799467861],720.166908,243.480632,50
3,771,1,271,[683.3949922251339],"[2.343074192693261, 1.9856719274571777, -0.583...",[5137.155016208713],782.751445,623.624035,50
4,774,1,274,[683.3950185919042],"[2.34307419269326, 1.9856719274571772, -0.5831...",[926.4774274383077],688.258098,34.031333,50


## DE 

In [11]:
# Definindo os parâmetros do de
pop_size = 50
genes = 7
generations = 300
lb = [-10]*7
ub = [10]*7
crossover_prob = 0.6
F = 0.8  # Fator de escala para a mutação
n_loops = 30  # Número de vezes que o algoritmo será executado

results_de = []
final_de = []

# Loop principal
for loop in range(n_loops):
    # Inicialização da população
    population = np.random.uniform(low=lb, high=ub, size=(pop_size, genes))
    for i in range(pop_size):
        while objective_function_with_constraints_g2(population[i]) == 0:
            population[i] = np.random.uniform(low=lb, high=ub, size=genes)

    # Loop das gerações
    for gen in range(generations):
        

        # Salvar o melhor indivíduo para elitismo
        fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
        # Salvar o melhor indivíduo para elitismo
        best_index = np.argmin(fitness_values)
        best_individual = population[best_index].copy()
        new_population = population.copy()

        for i in range(pop_size):
            indices = [idx for idx in range(pop_size) if idx != i]
            a, b, c = population[np.random.choice(indices, 3, replace=False)]
            mutant = np.clip(a + F * (b - c), lb, ub)  # Mutação
            cross_points = np.random.rand(genes) < crossover_prob
            if not np.any(cross_points):
                cross_points[np.random.randint(0, genes)] = True
            trial = np.where(cross_points, mutant, population[i])  # Crossover
            trial = np.clip(trial, lb, ub)  # Garantindo que o trial esteja dentro dos limites
            
            if objective_function_with_constraints_g2(trial) != 0:
                new_population[i] = trial

        population = new_population

        # Elitismo: garantir que o melhor indivíduo esteja na nova população
        new_fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
        worst_index = np.argmax(new_fitness_values)
        if new_fitness_values[worst_index] > fitness_values[best_index]:
            population[worst_index] = best_individual
        

        # Estatísticas da geração
        best_fitness = min(new_fitness_values)
        worst_fitness = max(new_fitness_values)
        mean_fitness = np.mean(new_fitness_values)
        std_dev_fitness = np.std(new_fitness_values)

        results_de.append({
            'loop': loop,
            'geracao': gen,
            'melhor_fitness': best_fitness,
            'melhor_particula': population[np.argmin(new_fitness_values)],
            'pior_fitness': worst_fitness,
            'media_dos_fitness': mean_fitness,
            'desvio_padrao': std_dev_fitness,
            'n_de_individuos': len(population),
        })
        print(f'Experimento {loop+1}, Geração {gen+1}')
        print(f'Melhor partícula: {population[np.argmin(new_fitness_values)]}, Fitness: {best_fitness}')
        print(f'Pior Fitness: {worst_fitness}')
        print('---'*20)
    final_de.append(best_fitness)

df_de = pd.DataFrame(results_de)



Streaming output truncated to the last 5000 lines.
Experimento 27, Geração 201
Melhor partícula: [ 2.11844005  1.18349678 -1.38611205  5.03703405 -0.05692422  0.39728923
  2.63738336], Fitness: [777.68972849]
Pior Fitness: [10011503.23808808]
------------------------------------------------------------
Experimento 27, Geração 202
Melhor partícula: [ 2.11844005  1.18349678 -1.38611205  5.03703405 -0.05692422  0.39728923
  2.63738336], Fitness: [777.68972849]
Pior Fitness: [10011358.22489625]
------------------------------------------------------------
Experimento 27, Geração 203
Melhor partícula: [ 2.11844005  1.18349678 -1.38611205  5.03703405 -0.05692422  0.39728923
  2.63738336], Fitness: [777.68972849]
Pior Fitness: [10011774.18654191]
------------------------------------------------------------
Experimento 27, Geração 204
Melhor partícula: [ 2.11844005  1.18349678 -1.38611205  5.03703405 -0.05692422  0.39728923
  2.63738336], Fitness: [777.68972849]
Pior Fitness: [10010812.23108803

In [12]:
df_de = df_de.sort_values('melhor_fitness', ascending=True).reset_index()
best_de_individual = df_de.loc[0, 'melhor_particula']
fitness_best_de_individual = df_de.loc[0, 'melhor_fitness']
df_de.head()

,index,loop,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,447,1,147,[702.6483013949104],"[1.1732115612250715, 1.9957794370243673, 0.466...",[10006009.030669592],1.904643e+06,3.589333e+06,50
1,516,1,216,[702.6483013949104],"[1.1732115612250715, 1.9957794370243673, 0.466...",[10011824.79401243],3.639616e+06,4.373782e+06,50
2,515,1,215,[702.6483013949104],"[1.1732115612250715, 1.9957794370243673, 0.466...",[10011464.936381806],3.330544e+06,4.381796e+06,50
3,514,1,214,[702.6483013949104],"[1.1732115612250715, 1.9957794370243673, 0.466...",[10011323.802228801],2.758543e+06,4.187480e+06,50
4,513,1,213,[702.6483013949104],"[1.1732115612250715, 1.9957794370243673, 0.466...",[10011351.21180278],3.216846e+06,4.391815e+06,50


##ES

In [ ]:
import numpy as np
from scipy.linalg import sqrtm

# Definindo os parâmetros da evolução estratégica
pop_size = 50
genes = 7
generations = 300
lb = [-10]*7
ub = [10]*7
sigma = 0.1  # Desvio padrão para a mutação
tau = 1/np.sqrt(2*genes)  # Parâmetro de autoadaptação

results_es = []
es_g_final = []



# Loop dos experimentos
for exp in range(30):
  # Inicialização da população
  particles = np.empty((pop_size, genes))
  for i in range(n_particles):  ## gerar individuos válidos 
      while True:
          particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
          if objective_function_with_constraints_g2(particle) != 0:
              particles[i] = particle
              break
  population = particles
  
  sigmas = sigma * np.ones((pop_size, genes, genes))
  # Loop das gerações
  
  for gen in range(generations):
      # Avaliação da população
      fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]

      # Elitismo: guardar o melhor indivíduo
      best_index = np.argmin(fitness_values)
      best_individual = population[best_index].copy()

      # Estratégia Evolutiva
      for i in range(pop_size):
          # Seleção de pais aleatórios
          parent_indices = np.random.choice(pop_size, 2, replace=False)
          parents = population[parent_indices]

          # CMA-ES
          mean = np.mean(parents, axis=0)
          cov = np.cov(parents.T)
          child = np.random.multivariate_normal(mean, cov)
          child = np.clip(child, lb, ub)

          # Adição de mutação
          child += sigma * np.random.randn(genes)
          child = np.clip(child, lb, ub)

          # Seleção (mu, sigma)
           # Seleção (mu, sigma)
          if constraints_g2(child) and objective_function_with_constraints_g2(child) < fitness_values[i]:
              population[i] = child

      # Elitismo: substituir o pior indivíduo pelo melhor da geração anterior, se necessário
      fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
      worst_index = np.argmax(fitness_values)
      if fitness_values[worst_index] > objective_function_with_constraints_g2(best_individual):
          population[worst_index] = best_individual

      # Estatísticas da geração
      best_fitness = min(fitness_values)
      worst_fitness = max(fitness_values)
      mean_fitness = np.mean(fitness_values)
      std_dev_fitness = np.std(fitness_values)

      results_es.append({
          'experimento': exp+1,
          'geracao': gen,
          'melhor_fitness': best_fitness,
          'melhor_particula': population[np.argmin(fitness_values)],
          'pior_fitness': worst_fitness,
          'media_dos_fitness': mean_fitness,
          'desvio_padrao': std_dev_fitness,
          'n_de_individuos': len(population),
      })

      # Imprime as melhores e piores partículas
      print(f'Experimento {exp+1}, Geração {gen+1}')
      print(f'Melhor partícula: {population[np.argmin(fitness_values)]}, Fitness: {best_fitness}')
      print(f'Pior partícula: {population[np.argmax(fitness_values)]}, Fitness: {worst_fitness}')
      print('---'*20)
      
  es_g_final.append(best_fitness)

df_g_es = pd.DataFrame(results_es)
df_g_es.head(5)



Streaming output truncated to the last 5000 lines.
  1.53739209], Fitness: [681.77867285]
------------------------------------------------------------
Experimento 2, Geração 155
Melhor partícula: [ 2.31110001  1.96344664 -0.14888691  4.31216013 -0.50523897  1.00668798
  1.53739209], Fitness: [681.25334767]
Pior partícula: [ 2.31110001  1.96344664 -0.14888691  4.31216013 -0.50523897  1.00668798
  1.53739209], Fitness: [681.77867285]
------------------------------------------------------------
Experimento 2, Geração 156
Melhor partícula: [ 2.31110001  1.96344664 -0.14888691  4.31216013 -0.50523897  1.00668798
  1.53739209], Fitness: [681.25334767]
Pior partícula: [ 2.31110001  1.96344664 -0.14888691  4.31216013 -0.50523897  1.00668798
  1.53739209], Fitness: [681.77867285]
------------------------------------------------------------
Experimento 2, Geração 157
Melhor partícula: [ 2.31110001  1.96344664 -0.14888691  4.31216013 -0.50523897  1.00668798
  1.53739209], Fitness: [681.25334767]


In [ ]:
df_es = df_g_es.sort_values('melhor_fitness', ascending=True).reset_index()
best_g_es_individual = df_g_es.loc[0, 'melhor_particula']
fitness_best_g_es_individual = df_g_es.loc[0, 'melhor_fitness']
df_es.head()

## resultados obtidos

In [ ]:
resultados_G2 = pd.DataFrame({
    'EVO. Alg.':['PSO','GA','DE','ES'],
    'Best Fitness':[fitness_best_PSO_individual,fitness_best_ga_individual,fitness_best_de_individual,fitness_best_g_es_individual],
    'Best Particle EA':[best_PSO_individual,best_ga_individual,best_de_individual,best_g_es_individual]})

resultados_G2 = resultados_G2.sort_values('Best Fitness',ascending=True).reset_index()
resultados_G2.head()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib as plt


# Aplanando os dados
de_final_flat = [np.asscalar(x) for x in final_de]
es_g_final_flat = [np.asscalar(x) for x in es_g_final]
ga_final_flat = [np.asscalar(x) for x in ga_final]

# Agora, todos os seus dados são listas simples de números
data = pd.DataFrame({
    'PSO': pso_final,
    'GA': ga_final_flat,
    'DE': de_final_flat,
    'ES G': es_g_final_flat,
})

plt.figure(figsize=(10, 6))
sns.boxplot(data=data)
plt.title('Boxplot dos Fitness Finais')
plt.ylabel('Fitness')
plt.show()

